# 1. Install and Import Dependencies

In [ ]:
!pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio===0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 & pip install -r requirements.txt

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import PIL
import torchvision.transforms as transforms

# 2. Load Model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

model

# 3. Make Detections with Images

In [ ]:
img = PIL.Image.open('C:/Users/yourm/Desktop/yunghab3/20220603_114217.png')
img.show()

In [ ]:
# object detection 적용된 이미지를 저장
results = model(img)
results.save()

# 4. Real Time Detections

In [ ]:
# 웹켐을 이용
cap = cv2.VideoCapture(0) #0은 웹캠을 의미함 만약 파일이나 웹주소라면 그대로 주소를 쓰면됨
while cap.isOpened():
    ret, frame = cap.read() # ret은 이미지 유무(True, Flase), frame 은 이미지
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'): # q 를 누르면 빠져나옴.
        break
cap.release()
cv2.destroyAllWindows()

# 5. Train from scratch

In [ ]:
import uuid   # Unique identifier
import os
import time

In [ ]:
IMAGES_PATH = os.path.join('yolov5/data', 'images') #/yolov5/data/images 디렉토리에 저장된다.
labels = ['awake', 'drowsy'] # 깨있는상태, 졸린상태 2가지 구분
number_imgs = 20  # 각 20장씩

In [ ]:
print(os.getcwd())
cap = cv2.VideoCapture(0)
# Loop through labels
for label in labels:  # 20장, 20장  총 40장을 찍는다.
    print('Collecting images for {}'.format(label))
    time.sleep(5) # 5초후에 시작
    
    # Loop through image range
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        # Webcam feed
        ret, frame = cap.read()
        
        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg') # 유니크한 파일이름을 생성해줌
        print('imgname : ',imgname)
        # Writes out image to file 
        cv2.imwrite(imgname, frame)
        
        # Render to the screen
        cv2.imshow('Image Collection', frame)
        
        # 2 second delay between captures
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# 라벨링하는 프로그램 설치
# labelImg 폴더위치 확인 필요..
!git clone https://github.com/tzutalin/labelImg

In [ ]:
# 원래 소스에는 !pip install pyqt5 lxml --upgrade  이거여지만 spyder와 버전문제로 아래처럼 수정하였슴.
print("###### spyder")
!pip install spyder --upgrade
print("###### pyqt5")
!pip install pyqt5 --upgrade
print("###### lxml")
!pip install lxml --upgrade



In [ ]:
# 현재 디렉토리 위치 확인하고 설치한 labelImg 폴더로 이동
print(os.getcwd())


In [ ]:
#%cd labelImg

In [ ]:
# resources.qrc 파일을 resources.py 파일로 바꿔준다고 합니다.
# labelImg.py 파일을 돌리기 위해필요
#아래는 윈도우에서 
!pyrcc5 -o libs/resources.py resources.qrc

#다른 OS는 https://github.com/tzutalin/labelImg 에서 확인후 실행

In [ ]:
# labelImg.py 파일 실행
print(os.getcwd())
!python labelImg.py

### 5-2 라벨만드는방법은 다음게시글을 확인하세요

### 5-3 트레이닝

In [ ]:
!python train.py --img 320 --batch 16 --epochs 20 --data dataset.yml --weights yolov5s.pt --workers 2

### 5-4 colab 에서 트레이닝은 다음게시글을 확인하세요

# 6. Load Custom Model

In [ ]:
# 우리가 만든 모델을 이용해서 테스트합니다.
import os
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
os.getcwd()
#%cd ..

In [ ]:
# path 에 train 시킨 모델을 적어줍니다.
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./yolov5/runs/train/exp_colab/weights/best.pt', force_reload=True)

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# 모델 변환

In [ ]:
### 1. Yolo v5(.pt model)  ---> .pb model
### 2. pb model  ---> .tflite model
### 3. Use tflite model in Android App

In [ ]:
import os
# 폴더위치 : yolov5 에서 실행
os.getcwd()


In [ ]:
#%cd yolov5

In [ ]:
# 같은 위치에 tflite 파일을 생성합니다. saved_model 폴더에 pb 파일도 생성됩니다.
!python export.py --weights ./runs/train/exp_colab/weights/best.pt --include tflite --img 416

In [ ]:
# 테스트 해봅니다.
# --data dataset.yml을 넣지않으면 coco128.yaml 로 실행될겁니다.
# source 는 웹캠으로 잡았습니다. 파일주소를 넣어도 됩니다.
!python detect.py --weights ./runs/train/exp_colab/weights/best-fp16.tflite --data dataset.yml --img 416 --conf 0.25 --source 0